In [1]:
import json
import os
import threading
import time

from kafka import KafkaProducer
from pyspark.sql import SparkSession

%load_ext autoreload
%autoreload 2

# Producing data
Now that Kafka and Zookeeper are running, we can start producing data.

In [2]:
spark = SparkSession.builder.appName("KafkaProducer").getOrCreate()

24/12/06 17:25:05 WARN Utils: Your hostname, Nikhils-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.206 instead (on interface en0)
24/12/06 17:25:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/06 17:25:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
producer = KafkaProducer(bootstrap_servers='localhost:9092', value_serializer=lambda v: json.dumps(v).encode('utf-8'))
topic = 'transactions'

In [4]:
data = spark.read.csv("./../dataset/creditcard.csv", header=True, inferSchema=True).cache()
data.show(5)

24/12/06 17:25:10 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+------------------+-------------------+----------------+------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+--------------------+-------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------+-----+
|Time|                V1|                 V2|              V3|                V4|                 V5|                 V6|                 V7|                V8|                V9|                V10|               V11|               V12|               V13|               V14|               V15|               V16|               V17|                V18|               V19|                V20|                 V21|                V22|     

- [x] Spark session is created
- [x] Kafka producer is created
- [x] Data is read from the CSV file
- [x] Data is shown

Now, we will send the data to the Kafka topic.

In [6]:
from pyspark.sql.functions import col
import math

# Assuming your original DataFrame is named 'df' and the class column is named 'Class'

# Separate the DataFrame into two classes
class_0 = data.filter(col("Class") == 0)
class_1 = data.filter(col("Class") == 1)

# Calculate the number of rows to sample from each class
rows_per_class = 25

# Sample from each class
class_0_sampled = class_0.sample(withReplacement=True, fraction=rows_per_class/class_0.count(), seed=42)
class_1_sampled = class_1.sample(withReplacement=True, fraction=rows_per_class/class_1.count(), seed=42)

# Union the sampled DataFrames
balanced_df = class_0_sampled.union(class_1_sampled)

# Verify the balance and total count
print(balanced_df.groupBy("Class").count().show())
print(f"Total rows: {balanced_df.count()}")

+-----+-----+
|Class|count|
+-----+-----+
|    0|   24|
|    1|   33|
+-----+-----+

None
Total rows: 57


In [7]:
counter = 0
for row in balanced_df.collect():
    print(f"Sending row {counter}")
    counter += 1
    producer.send(topic, value=row.asDict())
    time.sleep(1)
producer.flush()

Sending row 0
Sending row 1
Sending row 2
Sending row 3
Sending row 4
Sending row 5
Sending row 6
Sending row 7
Sending row 8
Sending row 9
Sending row 10
Sending row 11
Sending row 12
Sending row 13
Sending row 14
Sending row 15
Sending row 16
Sending row 17
Sending row 18
Sending row 19
Sending row 20
Sending row 21
Sending row 22
Sending row 23
Sending row 24
Sending row 25
Sending row 26
Sending row 27
Sending row 28
Sending row 29
Sending row 30
Sending row 31
Sending row 32
Sending row 33
Sending row 34
Sending row 35
Sending row 36
Sending row 37
Sending row 38
Sending row 39
Sending row 40
Sending row 41
Sending row 42
Sending row 43
Sending row 44
Sending row 45
Sending row 46
Sending row 47
Sending row 48
Sending row 49
Sending row 50
Sending row 51
Sending row 52
Sending row 53
Sending row 54
Sending row 55
Sending row 56
